# Profile Wizard: Interactive Classroom Setup

This notebook demonstrates the end-to-end workflow for setting up a new Classroom Profile, including:
1.  **Ingestion**: Loading a Pedagogical Guide (PDF).
2.  **Discovery**: Searching for relevant teaching strategies based on context (e.g., "Grade 5 English").
3.  **Refinement**: Interactively refining the system prompt.
4.  **Persistence**: Saving the final profile to the database.

## Setup
Ensure you have the required dependencies and environment variables (`OPENAI_API_KEY`, `POSTGRES_DB`, etc.) set.

In [ ]:
import os
import json
from typing import List

# Ensure we are in the root directory for imports
import sys
sys.path.append("../../")

from ingest.pedagogy_ingestor import PedagogyIngestor
from app.services.pedagogy_service import get_pedagogy_service
from app.services.profile_service import get_profile_service
from app.schemas import TeacherProfile, PedagogyConfig, ContentScope

print("Imports successful.")

## 1. Ingest Pedagogical Guide
We will ingest a sample PDF guide. In a real scenario, this happens once by an admin.
Here, we ingest `data/ESL_1st_guide_german.pdf`.

In [ ]:
ingestor = PedagogyIngestor()
pdf_path = "../../data/ESL_1st_guide_german.pdf"

if os.path.exists(pdf_path):
    print(f"Ingesting {pdf_path}...")
    try:
        strategy_id = ingestor.ingest_guide(pdf_path, title_override="Official ESL Guide (Grade 5-10)")
        print(f"Successfully ingested strategy. ID: {strategy_id}")
    except Exception as e:
        print(f"Ingestion failed (DB might be missing or OpenAI key invalid): {e}")
else:
    print(f"File {pdf_path} not found. Skipping ingestion.")

## 2. Wizard Step 1: Define Context
The teacher defines their class context.

In [ ]:
# User Input
teacher_input = {
    "class_name": "5A English",
    "subject": "English",
    "grade": 5,
    "goal": "Focus on speaking and active vocabulary."
}

print(f"Teacher Context: {teacher_input}")

## 3. Wizard Step 2: Discover Strategies
The system searches the `pedagogy_strategies` database for relevant guides.

In [ ]:
pedagogy_service = get_pedagogy_service()

try:
    strategies = pedagogy_service.search_strategies(
        query=teacher_input["goal"],
        subject=teacher_input["subject"],
        grade=teacher_input["grade"]
    )
    
    print(f"Found {len(strategies)} relevant strategies:")
    for s in strategies:
        print(f"- {s.title} (Relevance: High)")
        
    # Synthesize Initial Prompt
    initial_system_prompt = pedagogy_service.generate_pedagogy_prompt(strategies)
    print("\n--- DRAFT SYSTEM PROMPT ---")
    print(initial_system_prompt)
    
except Exception as e:
    print(f"Search failed (DB connectivity issue?): {e}")
    initial_system_prompt = "Default: Be a helpful teacher."

## 4. Wizard Step 3: Interactive Refinement
The user can "chat" to tweak the prompt before saving. (Simulated here)

In [ ]:
# Simulated User Feedback
feedback = "Make it more strict on grammar mistakes."
print(f"User Feedback: {feedback}")

# Simple rule-based update for demo (in prod, this would be another LLM call)
final_prompt = initial_system_prompt + "\n\n**ADDITIONAL USER PREFERENCE:**\n" + feedback

print("\n--- FINAL SYSTEM PROMPT ---")
print(final_prompt)

## 5. Wizard Step 4: Save Profile
Persist the profile to `teacher_profiles`.

In [ ]:
profile_service = get_profile_service()

new_profile = TeacherProfile(
    user_id="user_123",
    name=teacher_input["class_name"],
    grade_level=str(teacher_input["grade"]),
    pedagogy_config=PedagogyConfig(
        tone="encouraging",
        style="interactive",
        focus_areas=["vocabulary", "speaking"],
        adaptation_level="standard"
    ),
    content_scope=ContentScope()
)

# In a real app, we might store the 'final_prompt' in a special field or override 'pedagogy_config' 
# For now, we save the standard config.

try:
    saved_profile = profile_service.create_profile(new_profile)
    print(f"Profile saved successfully! ID: {saved_profile.id}")
except Exception as e:
    print(f"Failed to save profile: {e}")